In [105]:
from smb.SMBConnection import SMBConnection
import socket
import pathlib
import yaml
import os
import shutil

In [3]:
# Loading credentials
desktop = pathlib.Path.home() / 'Desktop'
conf = yaml.safe_load(open(str(desktop)+'/config.yml'))
usr = conf['user']['username']
pwd = conf['user']['password']
remote_address = conf['user']['remote_address']
domain = conf['user']['domain']

In [100]:
# LOCAL NAME
local_name = socket.gethostbyname(socket.gethostname())
# SERVER NAME
share_name = 'share'
server_name, alias, addresslist = socket.gethostbyaddr(remote_address)

In [116]:
folder_path = '/Test/test_dir/'
# Creating a local directory in the local computer
temp_local_path = pathlib.Path().absolute().joinpath('temp')
if os.path.exists(str(temp_local_path)):
    shutil.rmtree(temp_local_path)
os.makedirs(str(temp_local_path))
# Establishing connection to the server
conn = SMBConnection(username=usr, password=pwd, domain=domain, my_name=local_name, remote_name=server_name, is_direct_tcp=True)
is_connected = conn.connect(server_name,timeout=60)
print('Connection established: ', is_connected)
print('The files are downloaded to:', temp_local_path)

# Iterate in the folder to download all tif files
list_dir = conn.listPath(share_name, folder_path)
for file in list_dir:
    if (file.filename not in ['.', '..']) and ('.tif' in file.filename) :
        print ('File Downloaded :', file.filename)
        fileobj = open(file.filename,'wb')
        conn.retrieveFile(share_name,folder_path+file.filename,fileobj)
        # moving files in the local computer
        shutil.move(pathlib.Path().absolute().joinpath(file.filename), temp_local_path.joinpath(file.filename))
conn.close()

Connection established:  True
The files are downloaded to: /home/luisub/Desktop/FISH_Processing/notebooks/temp
File Downloaded : Capture_1_XY1617218260_Z00_T0_C0.tif
File Downloaded : Capture1_XY1617218260_Z00_T0_C2.tif
File Downloaded : Capture1_XY1617218260_Z00_T0_C1.tif


In [ ]:
# https://github.com/humberry/smb-example/blob/master/smb-test.py